In [1]:
import matplotlib
import csv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style, colors
from matplotlib import gridspec

matplotlib.style.use('ggplot')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (8, 6)

freeze list 123 patients: HTMCP-03-06-02061, squamous, hiv -, PIK3CA mutated' should exclude this sample
removed two hyper mutated cases:['HTMCP-03-06-02007', 'HTMCP-03-06-02026', 'HTMCP-03-06-02061']

In [2]:
def unique(a):
    """ return the list with duplicate elements removed """
    return list(set(a))

def intersect(a, b):
    """ return the intersection of two lists """
    return list(set(a) & set(b))

def union(a, b):
    """ return the union of two lists """
    return list(set(a) | set(b))

def exclusive(a, b):
    """ return items either exclusive in a or b """
    return list(set(a) ^ set(b))

def unique2a(a, b):
    """ return items only in a, but not in b """
    return list(set(a) & set(set(a) ^ set(b)))

def write(lst, of):
    with open(of, "w") as output:
        writer = csv.writer(output, lineterminator='\n')
        for val in lst:
            writer.writerow([val]) 

# look at NCI cervical snvs and indes

In [86]:
# snv summary file
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/HIGH_MODERATE_SNV_summary_with_normal_with_AF_filtered_somatic.tsv'
df1 = pd.read_csv(f, sep='\t', usecols=['gene', 'patient_ID'])
df1.head(2)

# indel summary file
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/HIGH_MODERATE_INDEL_summary_with_normal_strelka_indels_only.tsv'
df2 = pd.read_csv(f, sep='\t', usecols=['gene', 'patient_ID'])
df2.head(2)

dfc = pd.concat([df1, df2])
dfc['patient'] = dfc.patient_ID.str.replace('_malignant', '')
dfc.drop('patient_ID', inplace=True, axis=1)

dfc[dfc.patient.isin(['HTMCP-03-06-02061'])].head(2)

,gene,patient_ID
0,SAMD11,HTMCP-03-06-02007_malignant
1,SAMD11,HTMCP-03-06-02214_malignant


In [7]:
df1.shape

(44053, 2)

In [8]:
df2.shape 

(827, 2)

In [9]:
dfc.shape

(44880, 2)

In [10]:
dfc.head(2)

,gene,patient
0,SAMD11,HTMCP-03-06-02007
1,SAMD11,HTMCP-03-06-02214


In [90]:
dfc.gene.nunique()

14624

In [89]:
dfc = dfc.drop_duplicates()

In [13]:
df_allmmut = dfc.groupby('gene').count()

df_allmmut.shape

df_allmmut[df_allmmut.patient >=3].shape

(4832, 1)

In [110]:
df_allmmut.head(2)

,patient
gene,
A1BG,1
A1CF,2


In [15]:
allmmut_genes = df_allmmut.index.tolist()

remove hyper_mutated cases

In [91]:
x_patients = ['HTMCP-03-06-02007', 'HTMCP-03-06-02026', 'HTMCP-03-06-02061'] # 2 hyper-mutated cases, 1 previous maligancy

In [92]:
dfc = dfc[~dfc['patient'].isin(x_patients)]

In [93]:
dfc.shape

(22221, 2)

In [94]:
dfd = dfc.drop_duplicates()

In [95]:
dfe = dfd.groupby('gene').count()

In [96]:
dfe.head(2)

,patient
gene,
A1BG,1
A1CF,2


In [97]:
dfe.shape

(10972, 1)

In [99]:
# dfe.sort_values('patient')

In [101]:
fe = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutation_frequency.txt'
dfe.to_csv(fe, index=True, sep='\t')

In [21]:
df_minus3pat = dfe.copy()

In [22]:
minus3_genes = df_minus3pat.index.tolist()

In [23]:
dfe.index.nunique()

10972

In [24]:
dff = dfe[dfe.patient >= 3].sort_values('patient')

In [25]:
dff.head()

,patient
gene,
hsa-mir-4763,3
HSPA6,3
HSP90AA1,3
HS6ST1,3
HS3ST5,3


In [26]:
dff.shape

(2462, 1)

In [27]:
genes_3patients_minus3 = dff.index.tolist()

In [59]:
cols = ['gene', 
'patient_ID', 
'n_DNA_cov_valid_bases', 
'n_DNA_RefC', 
'n_DNA_AltC', 
'n_DNA_AF', 
't_DNA_cov_valid_bases', 
't_DNA_RefC', 
't_DNA_AltC', 
't_DNA_AF', 
'DNA_tc', 
't_RNA_cov_valid_bases', 
't_RNA_RefC', 
't_RNA_AltC', 
't_RNA_AF', 
'Quality_filter', 
'Somatic_filter', 
'filtered_somatic_num_patients_gene_level', 
'filtered_somatic_num_SNVs_gene_level', 
'filtered_somatic_num_patients_SNV_level']

In [34]:
ff = '/projects/trans_scratch/validations/workspace/szong/Cervical/2462_genes.txt'
pd.DataFrame(genes_3patients_minus3).to_csv(ff, index=False)

# look at epigenetic regulators

In [35]:
f3 = '/projects/trans_scratch/validations/workspace/szong/Cervical/epigenetics/143_epigentic_regulator_genes.txt'
df3 = pd.read_csv(f3, header=None)
df3.head()

,0
0,ASH2L
1,BAZ1A
2,BAZ1B
3,BRD2
4,BRD3


In [36]:
epi_genes = df3[0].tolist()

In [41]:
len(epi_genes)

143

In [46]:
# epi_genes_1patient = intersect(epi_genes, allmmut_genes)
epi_genes_1patient = intersect(epi_genes, minus3_genes)

In [47]:
len(epi_genes_1patient)

93

In [48]:
ueg = unique2a(epi_genes_1patient, genes_3patients_minus3)

In [49]:
len(ueg)

49

In [67]:
fg = '/projects/trans_scratch/validations/workspace/szong/Cervical/49_epi_genes.txt'
pd.DataFrame(ueg).to_csv(fg, index=False)

In [40]:
len(unique2a(epi_genes, minus3_genes))

50

# look at genes in spliceosome pathway

In [52]:
f4 = '/projects/trans_scratch/validations/workspace/szong/Cervical/spliceosome/240_spliceosome_genes.txt'
df4 = pd.read_csv(f4, sep='\t', usecols=['Approved Symbol'])
df4.head()

,Approved Symbol
0,CASC3
1,EIF4A3
2,MAGOH
3,RBM8A
4,LSM1


In [53]:
splice_genes = df4['Approved Symbol'].tolist()

In [59]:
aa = intersect(splice_genes, minus3_genes)


In [60]:
# aa  = intersect(splice_genes, allmmut_genes)

In [61]:
usg = unique2a(aa,genes_3patients_minus3 )


In [62]:
len(usg)

58

In [68]:
fh = '/projects/trans_scratch/validations/workspace/szong/Cervical/58_spliceosome_genes.txt'
pd.DataFrame(usg).to_csv(fh, index=False)

In [63]:
# genes_3patients_minus3, unique epigenetic genes, and unique splicing genes
len(set(usg + ueg + genes_3patients_minus3))

2569

In [78]:
mut_splicing_epi_genes = usg + ueg + genes_3patients_minus3

In [31]:
df4.columns

Index(['HGNC ID', 'Approved Symbol', 'Approved Name', 'Status',
       'Previous Symbols', 'Synonyms', 'Chromosome', 'Accession Numbers',
       'RefSeq IDs', 'Gene Family Tag', 'Gene family description',
       'Gene family ID'],
      dtype='object')

In [31]:
of2 = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients/tcga_genes_mutated_in_3_or_mroe_patients_not_recurrent_in_NCI.txt'
write(unique2a(a, b), of2)

# look at TCGA mutations

In [69]:
cols = ['Hugo_Symbol', 'trv_type','Tumor_Sample_Barcode']
f21 = '/projects/trans_scratch/validations/workspace/szong/Cervical/tcga_variants/tcga.genome.wustl.edu_CESC.IlluminaGA_DNASeq_curated.Level_2.1.0.0.somatic.maf'
df_tcga = pd.read_csv(f21, sep='\t', usecols=cols)
df_tcga.head(2)

,Hugo_Symbol,Tumor_Sample_Barcode,trv_type
0,A1BG,TCGA-BI-A0VS-01A-11D-A10S-08,missense
1,A1BG-AS1,TCGA-EK-A3GM-01A-11D-A20U-09,rna


In [70]:
df_tcga = df_tcga.drop_duplicates()

note TCGA included silent mutations

In [72]:
df_tcga = df_tcga.query('trv_type != "silent"')

In [73]:
df = df_tcga.groupby('Hugo_Symbol').count()

In [74]:
df.shape

(15006, 2)

In [75]:
df = df[df['Tumor_Sample_Barcode'] >=3]

In [76]:
tcga_genes_3patient = df.index.tolist()

In [79]:
len(tcga_genes_3patient)

4453

In [83]:
tcga_genes = unique2a(tcga_genes_3patient, mut_splicing_epi_genes)

In [85]:
fi = '/projects/trans_scratch/validations/workspace/szong/Cervical/3164_tcga_genes.txt'
pd.DataFrame(tcga_genes).to_csv(fi, index=False)

In [81]:
len(unique2a(tcga_genes_3patient, mut_splicing_epi_genes))

3164

In [103]:
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/2840_mut_freq.tmp'
df = pd.read_csv(f, sep='\t', header=None)
df.head()

,0,1
0,A2M,6
1,A4GALT,3
2,AAK1,3
3,ABCA1,6
4,ABCA12,3


In [104]:
f2 = '/projects/trans_scratch/validations/workspace/szong/Cervical/all_candidate_genes.txt'
df2 = pd.read_csv(f2, sep='\t', header=None)
df2.head()

,0,1
0,A2M,mutated_in_3_or_more_HTMCP_patient
1,A4GALT,mutated_in_3_or_more_HTMCP_patient
2,AAK1,mutated_in_3_or_more_HTMCP_patient
3,ABCA1,mutated_in_3_or_more_HTMCP_patient
4,ABCA12,mutated_in_3_or_more_HTMCP_patient


In [106]:
df3 = pd.merge(df, df2, left_on=0, right_on=0)

In [107]:
df3.shape

(2840, 3)

In [109]:
df3.columns = ['gene', 'number_HTMCP_patients_with_mutation', 'reason_picked_for_capture']

In [110]:
df3.head()

,gene,number_HTMCP_patients_with_mutation,reason_picked_for_capture
0,A2M,6,mutated_in_3_or_more_HTMCP_patient
1,A4GALT,3,mutated_in_3_or_more_HTMCP_patient
2,AAK1,3,mutated_in_3_or_more_HTMCP_patient
3,ABCA1,6,mutated_in_3_or_more_HTMCP_patient
4,ABCA12,3,mutated_in_3_or_more_HTMCP_patient


In [114]:
f3 = '/projects/trans_scratch/validations/workspace/szong/Cervical/all_candidate_validation_genes.txt'
df3.sort_values('number_HTMCP_patients_with_mutation', ascending=False).to_csv(f3, index=False, sep='\t')